# Project 4: Generating and Finetuning Transformer Language Models With Huggingface 

In this project, you will first learn how to use Huggingface's Transformers library to load large language models. Next, we will generate text from these models. Finally, we will finetune models on two tasks (sentiment analysis and machine translation).

This project will be more open ended than the previous projects. We expect you to learn how to use the huggingface and torch documentation.

## Setup

First we install and import the required dependencies. These include:
* `torch` for modeling and training
* `transformers` for pre-trained models
* `datasets` from huggingface to load existing datasets.

In [1]:
%%capture
!pip install transformers
!pip install datasets
!pip install --upgrade sacrebleu sentencepiece

# Standard library imports
import torch
from torch.utils.data import Dataset, random_split
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelWithLMHead

Before proceeding, let's verify that we're connected to a GPU runtime and that `torch` can detect the GPU.
We'll define a variable `device` here to use throughout the code so that we can easily change to run on CPU for debugging.

In [2]:
assert torch.cuda.is_available()
device = torch.device("cuda")
print("Using device:", device)

Using device: cuda


### Loading Model

We will use GPT-2 medium for this project. This includes both the GPT-2 tokenizer and the GPT-2 model weights itself. If you want to learn more about this model, you can read the GPT-2 paper https://d4mucfpksywv.cloudfront.net/better-language-models/language_models_are_unsupervised_multitask_learners.pdf.

Let's first load the tokenizer for the GPT-2 medium model. You can find how to do this by reading the documentation for AutoTokenzier in transformers, and finding the GPT-2 model of ~345 million params in there.

In [3]:
from transformers import AutoTokenizer
# Your code here
tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")

Let's tokenize and detokenize some text from this model.

In [4]:
print(tokenizer.encode('Hello world'))
print(tokenizer.decode(tokenizer.encode('Hello world')))
print(tokenizer.encode("Hola, cómo estás😍"))

[15496, 995]
Hello world
[39, 5708, 11, 269, 10205, 5908, 1556, 40138, 47249, 235]


Now let's load the GPT-2 medium model. Make sure you also put the model onto the GPU.

In [5]:
from transformers import AutoModelWithLMHead
# Your code here
gpt2_model = AutoModelWithLMHead.from_pretrained("gpt2-medium").to(device)

/opt/conda/lib/python3.7/site-packages/transformers/models/auto/modeling_auto.py:1252: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


## Generate From the Model

Now let's generate some text from the model to test its LM capabilities. Let's generate 10 pieces of random text of length 50 tokens from the model using random sampling with temperature set to 0.7. This will allow the text to be somewhat high in diversity (random sampling) while maintaining reasonable quality (temperature < 1). When generating text, you can condition on phrases such as "The coolest thing in NLP right now is". Find the relevant function and arguments to use for generating text using the Huggingface documentation.

Hint: you may find https://huggingface.co/docs/transformers/main_classes/text_generation to be useful for learning about generating from LMs.

In [6]:
inputs = tokenizer("<|startoftext|>The coolest thing right now in NLP is", return_tensors="pt").input_ids.cuda()
# Your code here
sample_outputs = gpt2_model.generate(inputs, do_sample=True, temperature=0.7, top_k=2, max_new_tokens=50)

Now lets print the text.

In [7]:
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

0: <|startoftext|>The coolest thing right now in NLP is the ability to use the "word" as a "word" in a sentence, and to use it as a word in a sentence. This is really cool, and I'm really excited about it. I'm also excited about the fact that I


Now generate one piece of text of length 50 with the same prompt ("The coolest thing right now in NLP is") but use greedy decoding (temperature = 0). This roughly corresponds to generating some text that is high likelihood for the model.

In [8]:
inputs = tokenizer("<|startoftext|>The coolest thing right now in NLP is", return_tensors="pt").input_ids.cuda()
# Your code here
sample_outputs = gpt2_model.generate(inputs, max_new_tokens=50)
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

0: <|startoftext|>The coolest thing right now in NLP is the ability to use the word "startoftext" to refer to a word that is not part of the text. This is useful for things like "the word" in a sentence, or "the word" in a sentence that is not part


Now let's try to see how good of a translation system GPT-2 medium is when used "out of the box". To accomplish this, we can condition on a prompt like the one below and generate from the model with greedy decoding. This will attempt to translate the sentence "UC Berkeley ist eine Schule in Kalifornien", which means "UC Berkeley is a school in California". Make sure to set the max length to be high enough so that the model generates sufficient text.

In [9]:
prompt = """Translate the following texts into English.

German: UC Berkeley ist eine Schule in Kalifornien
English:"""

In [10]:
# Your code here. Generate from the model using greedy decoding with the above prompt
tokens = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
sample_outputs = gpt2_model.generate(tokens, max_new_tokens=50)
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

0: Translate the following texts into English.

German: UC Berkeley ist eine Schule in Kalifornien
English: UC Berkeley ist eine Schule in Kalifornien

English: UC Berkeley ist eine Schule in Kalifornien

English: UC Berkeley ist eine Schule in Kalifornien

English


\As we can see, translation quality is terrible, as it just repeats the words from the previous text.

Now, let's finetune GPT-2 on the translation task to improve the results. We will use a translation dataset from the Huggingface dataset repository (it has thousands of other datasets available). This dataset is one of TED talks translated between German and English.

In [11]:
import datasets
dataset = datasets.load_dataset("ted_talks_iwslt", language_pair=("de", "en"), year="2014")

Generating train split: 0 examples [00:00, ? examples/s]

Dataset ted_talks_iwslt downloaded and prepared to /root/.cache/huggingface/datasets/ted_talks_iwslt/de_en_2014-c6e771351acd148b/1.1.0/43935b3fe470c753a023642e1f54b068c590847f9928bd3f2ec99f15702ad6a6. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
tokenizer.pad_token = tokenizer.eos_token

In [13]:
print(dataset['train'][0]['translation'])

{'de': '"Ich habe Zerebralparese. Ich zappele die ganze Zeit", kündigt Maysoon Zayid zu Anfang dieses ungeheuer witzigen, erheiternden an. (Er ist wirklich ungeheur witzig.) "Als würde Shakira auf Muhammad Ali treffen." Elegant und scharfsinnig nimmt uns die arabisch-amerikanische Komikerin auf eine Reise durch ihre Abenteuer als Schauspielerin, Komikerin, Philanthropin und Fürsprecherin für Menschen mit Behinderungen mit.', 'en': '"I have cerebral palsy. I shake all the time," Maysoon Zayid announces at the beginning of this exhilarating, hilarious talk. (Really, it\'s hilarious.) "I\'m like Shakira meets Muhammad Ali." With grace and wit, the Arab-American comedian takes us on a whistle-stop tour of her adventures as an actress, stand-up comic, philanthropist and advocate for the disabled.'}


Now we can create a dataset. For each element in the dataset, it should have a text prompt and then the translation, similar to above. Your job is to fill in the labels field below. This field sets the labels to use for training during the language modeling task. 

For the labels, we only want to train the model to output the text after the words "English:". This is because in the prompt, everything before the words "English:" will also be provided to the model as input. Hint: use -100 as the label for tokens you do not want to train on.
Hint 2: When doing LM training, the labels are the same as the input tokens, except shifted to the left by one. You should check whether Huggingface is already doing the shifting, or whether you need to do the shifting yourself.

One thing to be careful of with all LMs is to make sure there are not extra spaces. So, the text should be formatted as like "English: Hello..." not "English:  Hello...". This issue is a common problem people face when using APIs like GPT-3 which we will cover next time.

In [14]:
prompt = """Translate the following texts into English.
German: """

class TranslationDataset(Dataset):
    def __init__(self, examples, tokenizer):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for example in examples:
            training_text = prompt + example['translation']['de'] + '\nEnglish: ' + example['translation']['en'] + "<|endoftext|>"
            encodings_dict = tokenizer(training_text, max_length=275, padding="max_length", truncation=True)
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
            prompt_and_input_length = len(tokenizer.encode(prompt + example['translation']['de'] + '\nEnglish:'))
            # your code below
            label = torch.tensor([-100]*prompt_and_input_length + encodings_dict['input_ids'][prompt_and_input_length:])
            self.labels.append(label)


    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {'input_ids':self.input_ids[idx], 'attention_mask':self.attn_masks[idx], 'labels':self.labels[idx]}

In [15]:
translation_dataset = TranslationDataset(dataset['train'], tokenizer)

Now let's break the dataset into a train and test split.

In [16]:
train_size = int(0.9 * len(translation_dataset))
train_dataset, val_dataset = random_split(translation_dataset, [train_size, len(translation_dataset) - train_size])
print(len(train_dataset))
print(len(val_dataset))

2674
298


In [17]:
print(train_dataset[0])

{'input_ids': tensor([ 8291, 17660,   262,  1708, 13399,   656,  3594,    13,   198, 16010,
           25,  9956,    77,   519, 32188, 19136,  7802,  1931,    74,   917,
          316,  4656, 15942,  2398,   268,   268,  1471,   660,   287,  4587,
          356,   270,   268,   370,  2120,   851, 42499,   287, 10564,   368,
          304,   521,  1806,   677,   831, 18132,    89,    85,   419, 22562,
          329,    67,   861,  1931,  5576,   257,  3046,    11,   304,   259,
        14853,   444,   287,   384,  7749,   367,  3849, 39891,   299,   321,
          641, 18687, 27541,  4763,  1976,    84,   302, 32407,    13,   220,
         6733, 17380,  7123, 41555,    11,   450,   469,  1455,   268,  3318,
          555,   527,  9116, 11840,    83,    11,   264,   521,   287,   402,
          891,   993,    81,    11,   356,   346,   264,   494,   302, 14234,
         1665,   364, 40004,   307,   372,  3900,   268,    13, 11707,   379,
         1491,    64,   549,   437,   268,   376, 

Now we can use the Huggingface Trainer to finetune GPT-2 on this dataset. This abstracts away all of the details of training. Setup the training arguments to perform 3 epochs of training on this dataset, use a per-device batch size of 2 with gradient accumulation set to 8, use 100 warmup steps, a weight decay of 0.05. Set the eval batch size to be 2. Save a checkpoint every 250 steps. Set fp16 to True. Save the checkpoint in a specific output_dir so you can load it later. Hint: if it tries to launch Wandb, you may add the argument report_to="none".

In [18]:
# Your code here
training_args = TrainingArguments(
    output_dir='./data',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    warmup_steps=100,
    weight_decay=0.05,
    per_device_eval_batch_size=2,
    save_steps=1,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    fp16=True,
    report_to="none"
)

Next create a Huggingface Trainer object and call train() on it.

In [19]:
# Your code here
trainer = Trainer(
    gpt2_model,
    training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss
0,No log,0.477291
1,No log,0.441537
2,0.857900,0.440185


TrainOutput(global_step=501, training_loss=0.8569660044001962, metrics={'train_runtime': 1208.7733, 'train_samples_per_second': 6.636, 'train_steps_per_second': 0.414, 'total_flos': 4000487073792000.0, 'train_loss': 0.8569660044001962, 'epoch': 3.0})

Now load your saved checkpoint and see how well the finetuned GPT-2 model does on translating the sentence from before.

In [39]:
# Jim and Carey: The secret to being happy

prompt = """Translate the following texts into English.

German: Jim und Carey: Das Geheimnis des Glücklichseins
English:"""

# your code here
model = AutoModelWithLMHead.from_pretrained('./data/checkpoint-501').to(device)
tokens = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
sample_outputs = model.generate(tokens, max_new_tokens=50)
print(tokenizer.decode(sample_outputs[0], skip_special_tokens=True))

Translate the following texts into English.

German: Jim und Carey: Das Geheimnis des Glücklichseins
English: Jim and Carey: The secret to happiness


In [40]:
# Arthur Bailey: Can technology solve all of our problems?

prompt = """Translate the following texts into English.

German: Arthur Bailey: Kann Technologie all unsere Probleme lösen?
English:"""

# your code here
model = AutoModelWithLMHead.from_pretrained('./data/checkpoint-501').to(device)
tokens = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
sample_outputs = model.generate(tokens, max_new_tokens=50)
print(tokenizer.decode(sample_outputs[0], skip_special_tokens=True))

Translate the following texts into English.

German: Arthur Bailey: Kann Technologie all unsere Probleme lösen?
English: Arthur Bailey: Can technology solve our problems?


In [41]:
# After that they sometimes go shopping.

prompt = """Translate the following texts into English.

German: Danach gehen sie manchmal noch einkaufen
English:"""

# your code here
model = AutoModelWithLMHead.from_pretrained('./data/checkpoint-501').to(device)
tokens = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
sample_outputs = model.generate(tokens, max_new_tokens=50)
print(tokenizer.decode(sample_outputs[0], skip_special_tokens=True))

Translate the following texts into English.

German: Danach gehen sie manchmal noch einkaufen
English: Danelle: How to make your own food


In [42]:
# I am pleased to meet you.

prompt = """Translate the following texts into English.

German: Es freut mich, dich kennenzulernen
English:"""

# your code here
model = AutoModelWithLMHead.from_pretrained('./data/checkpoint-501').to(device)
tokens = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
sample_outputs = model.generate(tokens, max_new_tokens=50)
print(tokenizer.decode(sample_outputs[0], skip_special_tokens=True))

Translate the following texts into English.

German: Es freut mich, dich kennenzulernen
English: David Deutsch: How to make free love


In [43]:
# A group of men are loading cotton onto a truck

prompt = """Translate the following texts into English.

German: Eine Gruppe von Männern lädt Baumwolle auf einen Lastwagen
English:"""

# your code here
model = AutoModelWithLMHead.from_pretrained('./data/checkpoint-501').to(device)
tokens = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
sample_outputs = model.generate(tokens, max_new_tokens=50)
print(tokenizer.decode(sample_outputs[0], skip_special_tokens=True))

Translate the following texts into English.

German: Eine Gruppe von Männern lädt Baumwolle auf einen Lastwagen
English: A group of young men in Germany build a last-mile bridge


If training went correctly, you should see a reasonable translation of the sentence, with some errors.

For the project report, find two sentences where the model succeeds and two sentences where the model fails. Describe what might be causing these types of failures.

Finally, revisit the code from project 2 on using and running the Multi30k dataset. Your goal will be to translate the test set using the GPT-2 model you just finetuned. You will then submit your test predictions as a txt file, where you place your model's prediction for each test example on a separate line. Feel free to copy and paste any code from HW2 that may be useful. Submit the file named as mt_predictions.txt to gradescope.

The GPT-2 model may not work that well on the Multi30k dataset, because there is a distribution shift where the Multi30k data looks different than the Ted talks data that you finetuned the model on. The takeaway I want people to have is that a general-purpose LM system can be decent at a task like translation, however, if you create a domain-specific model like a LSTM trained specifically on Multi30k, you can outperform the general purpose model.

For the project report, compare two translations from the GPT-2 versus LSTM model. Which one works better?

Hint: One failure mode for GPT-2 is that it may generate fluent sentences that are actually unrelated to the input.

In [ ]:
# Your code for generating mt_predictions.txt below
!pip install torchtext==0.6

In [ ]:
import torchtext

In [ ]:

extensions = [".de", ".en"]
source_field = torchtext.data.Field(tokenize=lambda x: x)
target_field = torchtext.data.Field(tokenize=lambda x: x)
training_data, validation_data, test_data = torchtext.datasets.Multi30k.splits(
    extensions, [source_field, target_field], root=".")

In [ ]:
print("Number of training examples:", len(training_data))
print("Number of validation examples:", len(validation_data))
print("Number of test examples:", len(test_data))
print()

for example in training_data[:10]:
  print(example.src)
  print(example.trg)
  print()

In [46]:
from tqdm.notebook import tqdm_notebook

prompt = """Translate the following texts into English.
German: """
translations = []
for i in tqdm_notebook(range(len(test_data))):
    example = test_data[i]
    src = example.src
    text = prompt + src + '\nEnglish:'
    tokens = tokenizer(text, return_tensors="pt").input_ids.cuda()
    sample_outputs = model.generate(tokens, max_new_tokens=50)
    text_output = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
    translations.append(text_output[len(text)+1:])
translations = '\n'.join(translations)
with open('mt_predictions.txt', 'w') as file:
    file.write(translations)
    

  0%|          | 0/1000 [00:00<?, ?it/s]

### Sentiment Analysis

The beauty of language models is that we can apply this exact same machinery to solve a completely different task of sentiment analysis. Here, we will be given a movie review and the goal is to have the model predict whether the review is positive or negative.

First, we will load some sentiment analysis data. Your job is to copy what we did above for machine translation to load the dataset, build a Class to create the dataset, etc., 

When doing so, use the prompt below, where you put the text of the input in the first [] and in the second [], put the word Positive if the label is 1 and the word Negative if the label is 0. Make sure to also set the self.labels field correctly, we only want to compute a loss on the words Positive/Negative, and no other tokens in the model's input.

The following is a movie review. [Movie Review Text Here]. The sentiment of the review is [Positive/Negative].

In [47]:
import datasets
dataset = datasets.load_dataset('glue', 'sst2')
print(dataset['train'][0])

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

{'sentence': 'hide new secretions from the parental units ', 'label': 0, 'idx': 0}


Note: Some people were saying that this line of code wasn't working and they needed to use "dataset = datasets.load_dataset('glue', 'sst2')" instead.

In [48]:
prompt = """The following is a movie review. """
class SentimentDataset(Dataset):
    def __init__(self, examples, tokenizer):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for example in examples:
            sentiment = 'Positive' if example['label'] else 'Negative'
            training_text = prompt + example['sentence'][:-1] + '. The sentiment of the review is ' + sentiment + ".<|endoftext|>"
            encodings_dict = tokenizer(training_text, max_length=275, padding="max_length", truncation=True)
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
            prompt_and_input_length = len(tokenizer.encode(prompt + example['sentence'][:-1] + '.The sentiment of the review is'))
            # your code below
            label = torch.tensor([-100]*prompt_and_input_length + encodings_dict['input_ids'][prompt_and_input_length:])
            self.labels.append(label)

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {'input_ids':self.input_ids[idx], 'attention_mask':self.attn_masks[idx], 'labels':self.labels[idx]}

In [49]:
sentiment_train_dataset = SentimentDataset(dataset['train'], tokenizer)
sentiment_val_dataset = SentimentDataset(dataset['validation'], tokenizer)

The data already comes with a validation and train split

In [50]:
print(len(sentiment_train_dataset))
print(len(sentiment_val_dataset))
print(sentiment_train_dataset[0])

67349
872
{'input_ids': tensor([  464,  1708,   318,   257,  3807,  2423,    13,  7808,   649,  3200,
          507,   422,   262, 21694,  4991,    13,   383, 15598,   286,   262,
         2423,   318, 36183,    13, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 5025

Now let's train the model using the same trainer arguments as before, except just do $<$1 epoch of training because this dataset is quite large and training on the entire thing will take some time. Make sure you also use a different output_dir so it doesn't overwrite your old results.

In [53]:
# Your code here
gpt2_model = AutoModelWithLMHead.from_pretrained("gpt2-medium").to(device)
training_args = TrainingArguments(
    output_dir='./sentiment',
    num_train_epochs=0.4,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    warmup_steps=100,
    weight_decay=0.05,
    per_device_eval_batch_size=2,
    save_steps=1,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    fp16=True,
    report_to="none"
)
trainer = Trainer(
    gpt2_model,
    training_args,
    train_dataset=sentiment_train_dataset,
    eval_dataset=sentiment_val_dataset,
    tokenizer=tokenizer
)
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss
0,0.000700,0.000806


TrainOutput(global_step=1684, training_loss=0.1422494201701922, metrics={'train_runtime': 3825.4557, 'train_samples_per_second': 7.042, 'train_steps_per_second': 0.44, 'total_flos': 1.34400403636224e+16, 'train_loss': 0.1422494201701922, 'epoch': 0.4})

At test-time, when you want to classify an incoming movie review, you can just check whether the model generates the words Positive or Negative as the final word.

In [75]:
prompt = """The following is a movie review. The acting was great but overall I was left disappointed by the film. The sentiment of the review is"""

In [76]:
# Your code here
model = AutoModelWithLMHead.from_pretrained('./sentiment/checkpoint-1684').to(device)
tokens = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
sample_outputs = model.generate(tokens, max_new_tokens=50)
print(tokenizer.decode(sample_outputs[0], skip_special_tokens=True))

The following is a movie review. The acting was great but overall I was left disappointed by the film. The sentiment of the review is Negative.


Finally, run the entire validation set through the model and get your model predictions. Save the results as a txt file, where each line just contains either "1" if your model predicted Positive and "0" if the model predicted Negative. You will get full credit if your model's accuracy is greater than 80%. Save the file as sst_predictions.txt and submit it to gradescope.

For the report, describe two possible improvements to your sentiment classifier.

In [118]:
# Your code here for generating sst_predictions
sentiment_val_dataset
prompt = """The following is a movie review. """
translations = []
for i in tqdm_notebook(range(len(dataset['validation']))):
    example = dataset['validation'][i]
    review = example['sentence'][:-1]
    text = prompt + review + '. The sentiment of the review is'
    tokens = tokenizer(text, return_tensors="pt").input_ids.cuda()
    sample_outputs = model.generate(tokens, max_new_tokens=50)[:, tokens.shape[1]:]
    text_output = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
    translations.append('1' if text_output[1:-1] == "Positive" else '0')
translations = '\n'.join(translations)
with open('sst_predictions.txt', 'w') as file:
    file.write(translations)

  0%|          | 0/872 [00:00<?, ?it/s]

## Submission

Turn in the following files on Gradescope:
* hw4.ipynb (this file; please rename to match)
* mt_predictions.txt (the predictions for the Multi30k test set)
* sst_predictions.txt (the predictions for the SST-2 validation set)
* report.pdf

Be sure to check the output of the autograder after it runs.  It should confirm that no files are missing and that the output files have the correct format.